In [15]:
import pandas as pd
import numpy as np
import seaborn as np
from matplotlib import pyplot as plt
import re

In [16]:
def extract_measurement(b_list):

    # this function extract the data points from B3,B4,B5 and 
    # And return clean data to get_measurment() in dictionay format 
    


    x_spos = b_list.index('LengthValue') # starting index of the x values
    y_spos = b_list.index('Value')# starting index of the y values
    y_epos = len(b_list)# the ending pos of y values
    x = b_list[x_spos + 1:y_spos-1]
    y = b_list[y_spos + 1: y_epos -2]
    xy_dict = dict(zip(x,y))
    print(len(xy_dict))

    # need some extra work to clean the data
    


    return xy_dict




def get_measurment(coil_id):

    # this function load the data for B3,B4,B5 measurments
    # It split the data based on ";"
    # It calls extract_measuremnt() function to extract the 
    # clean data pairs for B3,B4,B5
    # And return the values in dictionary format

    print('************************************')
    print("coil Id: ", coil_id)
    root_directory = "data/SampleProcessedData/"

    b3_df = pd.read_csv(root_directory + str(coil_id ) + "B3.csv")
    b4_df = pd.read_csv(root_directory + str(coil_id ) + "B4.csv")
    b5_df = pd.read_csv(root_directory + str(coil_id ) + "B5.csv")

    b3_list = re.split(';', str(b3_df.columns))
    b4_list = re.split(';', str(b4_df.columns))
    b5_list = re.split(';', str(b5_df.columns))

    b_lists = [b3_list,b4_list,b5_list]

    all_B_list = [] # this list contain all xy_dict for B3,B4,B5(it is list of dictionaries)
    for b_list in b_lists:
        all_B_list.append(extract_measurement(b_list))
        
        #print(b_dict)
        #print("###############################")


    print('*********************************')

    # use this code as starting code
    
    # for dictionary in all_B_list:
    #     for key in dictionary:
    #         dictionary[float(key)] = float(dictionary[key])
    
    return all_B_list # return list of dictionaries
    



    
 
    

    

In [17]:
def label_observation():

    # this function get the coil_Id and call the get_measurment()
    # function to get the cleaned value for B3,B4 and B5 in dictionary format
    # from the dictionary it checks if there is constriction or not
    # and add the result to the coil_df

     #load the dataset
     coil_df = pd.read_excel('data/CoilDataInput.xlsx')
     count = 0

     measurment = get_measurment(290044)
     print(measurment)


     # for name,coil_id in coil_df['coil'].items():
          
     #      count += 1
          
     #      if count == 4:
     #           exit()
     #      try:
     #           get_measurment(coil_id)
     #      except:
     #           print('not found')
     #           continue
          
          # write code to label observation


     

label_observation()
     


     


************************************
coil Id:  290044
1055
1054
1031
*********************************


RuntimeError: dictionary changed size during iteration

In [ ]:
def visualize_measurments():
    pass